## Load Modules

In [1]:
import pandas as pd
from wordcloud import STOPWORDS
import re
#import en_core_web_md
import spacy
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import numpy as np

## Read File

In [2]:
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-18_ProcessedDF.csv'
df = pd.read_csv(filename)

In [50]:
#create false negative col
def falseNegative(x):
    if len([1 for i in range(1, 8) if x['analystRule' + str(i)] and not x['rule' + str(i)]])!=0:
        return True
    else:
        return False
    
df['FN'] = df.apply(falseNegative, axis = 1)

#neither rule or analysts think should remove
def falsePositive(x):
    if len([1 for i in range(1, 8) if not x['analystRule' + str(i)] and x['rule' + str(i)]])!=0:
        return True
    else:
        return False
    
df['FP'] = df.apply(falsePositive, axis = 1)

def noFlag(x):
    if not x['FN'] and not x['FP'] and not x['TP']:
        return True
    else:
        return False
df['noFlag'] = df.apply(noFlag, axis = 1)

#store indices for bad str
badIndex = list(df[df['TP']].index)

processedStr = df['SUSPICION_DESC_CLEAN'].values

In [2]:
import pandas as pd
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-18_ProcessedDF.csv'
df = pd.read_csv(filename)

In [5]:
#store indices for bad str
badIndex = list(df[df['TP']].index)

#save as training data
dfTraining = df[df.index.isin(badIndex)].copy()
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-22_TrainingData.csv'
dfTraining.to_csv(filename, index = False)
dfTesting = df[~df.index.isin(badIndex)].copy()
filename = 'D://Users/figohjs/Documents/NLP/StrPrioritization/Data/Interim/2020-03-22_TestingData.csv'
dfTesting.to_csv(filename, index = False)

## Create dictionaries to map id to report and etc

In [5]:
IndexToReportID_Dict = df['RECORD_ID'].to_dict()

IndexToStrDesc_Dict = df['SUSPICION_DESC'].to_dict()

ReportIDToIndex_Dict = {j:i for i,j in IndexToReportID_Dict.items()}

In [74]:
IndexToReportID_Dict

{0: 'AC/011/S/2019/000040',
 1: 'AA/004/S/2019/005424',
 2: 'AA/010/S/2019/021501',
 3: 'AF/001/S/2019/000383',
 4: 'AC/014/S/2019/001532',
 5: 'AA/025/S/2019/000825',
 6: 'AA/020/S/2019/003371',
 7: 'AF/001/S/2019/000317',
 8: 'AC/014/S/2019/001354',
 9: 'AA/020/S/2019/002968',
 10: 'AA/025/S/2019/000874',
 11: 'AC/014/S/2019/001445',
 12: 'AA/021/S/2019/002852',
 13: 'AC/014/S/2019/001349',
 14: 'AA/025/S/2019/000806',
 15: 'AA/020/S/2019/003069',
 16: 'AC/014/S/2019/001444',
 17: 'AC/014/S/2019/001521',
 18: 'AC/004/S/2019/000527',
 19: 'AA/004/S/2019/005164',
 20: 'AA/018/S/2019/000951',
 21: 'AA/004/S/2019/005034',
 22: 'AA/021/S/2019/003411',
 23: 'AA/010/S/2019/019998',
 24: 'AA/020/S/2019/003306',
 25: 'AA/020/S/2019/003431',
 26: 'AA/010/S/2019/023789',
 27: 'AA/010/S/2019/019181',
 28: 'AC/014/S/2019/001370',
 29: 'AC/014/S/2019/001557',
 30: 'AA/020/S/2019/003311',
 31: 'AA/020/S/2019/003302',
 32: 'AC/014/S/2019/001365',
 33: 'AA/010/S/2019/023792',
 34: 'AA/004/S/2019/0055

## Model Training

In [6]:
#for neural network
maxEpoch = 5
vectorSize = 500
alpha = 0.025
minAlpha = alpha #no decreasing rate
minCount = 1
#training data
trainingData = [i.split(',')[0] for no, i in enumerate(processedStr) if no in badIndex]

In [7]:
class similarDoc():
    def __init__(self, **kwargs):
        #predefine prameters
        self.Doc2VecModel = Doc2Vec(**kwargs)
        self.MaxEpoch = 10
        self.ModelFolder = 'D:/Users/figohjs/Documents/NLP/StrPrioritization/Model'
        self.ModelName = self.ModelFolder + '/' + datetime.now().strftime('%Y-%m-%d') + '_' + 'Doc2Vec.model'
        #indices are based on list of tagged document
        self.BadStrIndex = kwargs['bad_index']
        self.Threshold = kwargs['threshold']
        self.VectorSize = kwargs['vector_size']
        self.SimilarBadDocList = []
        
    def prepareText(self, ContentList):
        #tag document
#         self.TaggedDocList = [TaggedDocument(doc, tags = [str(no)]) 
#                               for no, doc in enumerate([i.split() for i in ContentList])]
        self.TaggedDocList = [TaggedDocument(doc[1], tags = [doc[0]]) 
                              for doc in zip(self.BadStrIndex, [i.split() for i in ContentList])]
    def trainModel(self):
        #build vocab
        self.Doc2VecModel.build_vocab(self.TaggedDocList)
        for epoch in range(self.MaxEpoch):
            print('Iteration - %s'%epoch)
            self.Doc2VecModel.train(self.TaggedDocList,
                                    total_examples = self.Doc2VecModel.corpus_count,
                                    epochs = epoch)
            #self.Doc2VecModel.alpha -= 0.002
        self.Doc2VecModel.save(self.ModelName)
        print('%s is saved'%self.ModelName)
        
    def loadModel(self, TrainedModel):
        self.Doc2VecModel = gensim.models.doc2vec.Doc2Vec.load(TrainedModel)
                    
    def findSimilarDocForBadDoc(self, NewDoc, n = 10):
        NewVec = self.Doc2VecModel.infer_vector(NewDoc.split())
        similarRepList = []
        similarScoreList = []
        #loop through every bad report
        for index in self.BadStrIndex:
            similarScore = cosine_similarity(self.Doc2VecModel.docvecs[index].reshape(1, self.VectorSize),
                                             NewVec.reshape(1, self.VectorSize))
            #if similarity score exceeds threshold
            if similarScore >= self.Threshold:
                similarRepList.append(str(index))
                similarScoreList.append(str(similarScore[0][0]))
        #if not similar to any bad report, (a, b, c)
        #a - 1 if has similar to any bad report
        #b - list of reports 
        if len(similarRepList) == 0:
            return (0, '', '')
        else:
            return (1, ','.join(similarRepList), ','.join(similarScoreList))

In [8]:
#trainingData are all bad report, so badIndex is just np.arange(len)
#badIndex = np.arange(len(trainingData))
#store indices for bad str
badIndex = list(df[df['TP']].index)
#instantiate a class named similarDoc
paramDict = {'min_count': minCount, 'vector_size': vectorSize,
             'alpha':alpha, 'min_alpha':alpha, 'bad_index':badIndex,
            'threshold':0.75}

nlpSimilarity = similarDoc(**paramDict)

In [11]:
trainingData = [i.split(',')[0] for no, i in enumerate(processedStr) if no in badIndex]
nlpSimilarity.prepareText(trainingData)

nlpSimilarity.trainModel()

Iteration - 0
Iteration - 1
Iteration - 2
Iteration - 3
Iteration - 4
Iteration - 5
Iteration - 6
Iteration - 7
Iteration - 8
Iteration - 9
2020-03-20_Doc2Vec.model is saved


In [9]:
#load model
trainedModel = '2020-03-19_Doc2Vec.model'
nlpSimilarity.loadModel(trainedModel)

In [19]:
#find other reports that are similar
df2 = df[~df.index.isin(badIndex)].copy()

#use trained model to find if there is report similar with any bad reports
similarList = []
indexList = []
scoreList = []
for no in range(df2.shape[0]):
    info = str(df2['SUSPICION_DESC_CLEAN'].values[no])
    similarBool, index, score = nlpSimilarity.findSimilarDocForBadDoc(info)
    similarList.append(similarBool)
    indexList.append(index)
    scoreList.append(score)
pandaDict = {'similarBool':similarList,
            'reportIndexList':indexList,
            'scoreList':scoreList}
dfResult = pd.DataFrame(pandaDict)

dfResult = pd.concat([dfResult, df2.reset_index()], axis = 1)

dfResult.fillna('', inplace = True)

In [30]:
#get most similar report
def mostSimilarReport(indexArray, scoreArray):
    #store result
    resultList = []
    
    #loop through every row
    for no, tempArray in enumerate(scoreArray):
        #get max score from every list splitted by tempArray
        tempScoreList = str(tempArray).split(',')
        tempIndexList = str(indexArray[no]).split(',')
        maxScore = max(tempScoreList)
        indexList = [(i, tempScoreList[no]) for no, i in enumerate(tempIndexList) if  tempScoreList[no] == maxScore]
        
        if len(indexList) > 1:
            #take first element for interim
            resultList.append([indexList[0][0], indexList[0][1]])
        elif len(indexList) == 1:
            resultList.append([indexList[0][0], indexList[0][1]])
        else:
            resultList.append(['', ''])
          
    return pd.DataFrame(resultList, columns = ['mostSimilarIndex', 'mostSimilarScore'])

In [31]:
dfResult2 = mostSimilarReport(dfResult['reportIndexList'].values, dfResult['scoreList'].values)

colList = ['RECORD_ID', 'ANALYST', 'SUSPICION_DESC_CLEAN', 'TP', 'FN', 'FP', 'noFlag']
dfFinal = pd.concat([dfResult2, dfResult[colList].reset_index()], axis = 1).copy()

dfFinal['mostSimilarReportID'] = dfFinal['mostSimilarIndex'].map(lambda x:IndexToReportID_Dict[int(x)] 
                                                                 if str(x) not in ['', 'nan'] else '')

dfFinal['mostSimilarStrDesc'] = dfFinal['mostSimilarIndex'].map(lambda x:IndexToStrDesc_Dict[int(x)] 
                                                                 if str(x) not in ['', 'nan'] else '')

## Experiment

In [10]:
similarList = []
indexList = []
scoreList = []
for no in range(3):
    similarBool, index, score = nlpSimilarity.findSimilarDocForBadDoc(df['SUSPICION_DESC_CLEAN'].values[no])
    similarList.append(similarBool)
    indexList.append(index)
    scoreList.append(score)

In [38]:
pandaDict = {'similarBool':similarList,
            'reportIndexList':indexList,
            'scoreList':scoreList}
dfResult = pd.DataFrame(pandaDict)
dfResult = pd.concat([dfResult, df.iloc[:3]], axis = 1)
dfResult

,similarBool,reportIndexList,scoreList,RECORD_ID,ANALYST,Remove,Justification,SUSPICION_DESC,Free Text,analystRule1,...,analystRule7,rule1,rule2,rule3,rule4,rule5,rule6,rule7,TP,SUSPICION_DESC_CLEAN
0,1,"2,910,933,946,948,950,954,955,956,959,960,963,...","0.8562542,0.84573674,0.8081562,0.7795453,0.839...",AC/011/S/2019/000040,siashraf,No,amount too high,REFER ATTACHMENT,NaN,False,...,False,False,False,False,False,False,False,False,False,REFER ATTACHMENT
1,1,"2,910,932,933,946,948,950,954,955,958,959,960,...","0.86378956,0.87657577,0.7740998,0.7876052,0.76...",AA/004/S/2019/005424,siashraf,No,amount too high,REFER ATTACHMENT FOR FULL NARRATIVE,NaN,False,...,False,False,False,False,False,False,False,False,False,REFER ATTACHMENT FULL NARRATIVE
2,0,,,AA/010/S/2019/021501,siashraf,Yes,Investigation/Production/Freezing Order,STR RAISED PURSUANT TO THE INVESTIGATION OF TH...,NaN,True,...,False,True,False,False,False,False,False,False,True,STR RAISED PURSUANT INVESTIGATION CUSTOMER RMP...


In [52]:
#get most similar report
def mostSimilarReport(indexArray, scoreArray):
    #store result
    resultList = []
    
    #loop through every row
    for no, tempArray in enumerate(scoreArray):
        #get max score from every list splitted by tempArray
        tempScoreList = tempArray.split(',')
        tempIndexList = indexArray[no].split(',')
        maxScore = max(tempScoreList)
        indexList = [(i, tempScoreList[no]) for no, i in enumerate(tempIndexList) if  tempScoreList[no] == maxScore]
        
        if len(indexList) > 1:
            #take first element for interim
            resultList.append([indexList[0][0], indexList[0][1]])
        elif len(indexList) == 1:
            resultList.append([indexList[0][0], indexList[0][1]])
        else:
            resultList.append(['', ''])
          
    return pd.DataFrame(resultList)

In [53]:
mostSimilarReport(dfResult['reportIndexList'].values, dfResult['scoreList'].values)

,0,1
0,4197,0.9076031
1,4197,0.924866
2,,


## Evaluation

In [55]:
#success rate - flag how many FN
successNum = sum(np.logical_and((dfFinal['mostSimilarIndex']!='').values, dfFinal['FN'].values))
print("Success Rate : %s"%round((successNum * 100/sum(dfFinal['FN'])),2))

Success Rate : 10.11
